## !! PREPROCESSING.IPYNB로 이관했음, 추후 분리예정

In [94]:
import pandas as pd
import numpy as np


X_train = np.array(pd.read_csv('./data/mecab/X_train.csv'))
X_test = np.array(pd.read_csv('./data/mecab/X_test.csv'))
y_train_1 = np.array(pd.read_csv('./data/mecab/y_train.csv')['0'])
y_test_1 =  np.array(pd.read_csv('./data/mecab/y_test.csv')['0'])
y_train_2 = np.array(pd.read_csv('./data/mecab/y_train.csv')['1'])
y_test_2 =  np.array(pd.read_csv('./data/mecab/y_test.csv')['1'])
y_train_3 = np.array(pd.read_csv('./data/mecab/y_train.csv')['2'])
y_test_3 =  np.array(pd.read_csv('./data/mecab/y_test.csv')['2'])

print(len(X_train), len(X_test), len(y_train_1), len(y_test_1))

740000 185002 740000 185002


In [139]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
y_train_1 = le.fit_transform(y_train_1)
y_test_1 = le.fit_transform(y_test_1)


In [ ]:
y_train_2 = le.fit_transform(y_train_2)
y_test_2 = le.fit_transform(y_test_2)
y_train_3 = le.fit_transform(y_train_3)
y_test_3 = le.fit_transform(y_test_3)

In [48]:
from keras.utils import np_utils
y_train_1 = np_utils.to_categorical(y_train_1)
y_test_1 = np_utils.to_categorical(y_test_1)

y_train_2 = np_utils.to_categorical(y_train_2)
y_test_2 = np_utils.to_categorical(y_test_2)

y_train_3 = np_utils.to_categorical(y_train_3)
y_test_3 = np_utils.to_categorical(y_test_3)

In [78]:
y_train_1.argmax()

8

In [53]:
from tensorflow.keras.layers import Embedding, Dense, LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam

vocab_size = 19950
embedding_dim = 200
hidden_units = 128

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(LSTM(hidden_units))
model.add(Dense(19, activation='softmax'))

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
history = model.fit(X_train, y_train_1, epochs=10, callbacks=[es, mc], batch_size=128, validation_split=0.2)

Epoch 1/10
4625/4625 [==============================] - 376s 81ms/step - loss: 0.1429 - acc: 0.9612 - val_loss: 0.0888 - val_acc: 0.9751

Epoch 00001: val_acc improved from -inf to 0.97508, saving model to best_model.h5
Epoch 2/10
4625/4625 [==============================] - 378s 82ms/step - loss: 0.0719 - acc: 0.9793 - val_loss: 0.0841 - val_acc: 0.9758 0 - ETA: 1s - loss: 0.071

Epoch 00002: val_acc improved from 0.97508 to 0.97584, saving model to best_model.h5
Epoch 3/10
4625/4625 [==============================] - 381s 82ms/step - loss: 0.0589 - acc: 0.9825 - val_loss: 0.0859 - val_acc: 0.9761

Epoch 00003: val_acc improved from 0.97584 to 0.97611, saving model to best_model.h5
Epoch 4/10
4625/4625 [==============================] - 380s 82ms/step - loss: 0.0498 - acc: 0.9850 - val_loss: 0.0885 - val_acc: 0.9760

Epoch 00004: val_acc did not improve from 0.97611
Epoch 5/10
4625/4625 [==============================] - 273s 59ms/step - loss: 0.0426 - acc: 0.9873 - val_loss: 0.0919 -

In [54]:
loaded_model = load_model('best_model.h5')
print("\n 테스트 정확도: %.4f" % (loaded_model.evaluate(X_test, y_test_1)[1]))

5782/5782 [==============================] - 18s 3ms/step - loss: 0.0843 - acc: 0.9759

 테스트 정확도: 0.9759


In [76]:
loaded_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 200)         3990000   
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               168448    
_________________________________________________________________
dense_1 (Dense)              (None, 19)                2451      
Total params: 4,160,899
Trainable params: 4,160,899
Non-trainable params: 0
_________________________________________________________________


In [70]:
import pandas as pd
import numpy as np
import re

import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

from eunjeon import Mecab
from tqdm import tqdm # 진행상황 Progress Bar를 위한 tqdm library
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.math import argmax

In [88]:
# 한국표준산업분류 딕셔너리 생성

cat_dic = pd.read_excel('./data/한국표준산업분류(10차)_국문.xlsx').iloc[2:,0:6]
cat_dic.rename(columns={'개정 분류체계(제10차 기준)':'대분류', 'Unnamed: 1':'대분류명', 
                        'Unnamed: 2': '중분류', 'Unnamed: 3': '중분류명',
                        'Unnamed: 4': '소분류', 'Unnamed: 5': '소분류명'}, inplace=True)

digit_1_df = cat_dic[['대분류', '대분류명']].dropna().reset_index(drop=True) # 대분류 'A' ~ 'U'
digit_2_df = cat_dic[['중분류', '중분류명']].dropna().reset_index(drop=True) # 중분류 1 ~ 99
digit_3_df = cat_dic[['소분류', '소분류명']].dropna().reset_index(drop=True) # 소분류 11 ~ 990

digit_1_dict = {}
for i in tqdm(range(len(digit_1_df))):
    digit_1_dict[digit_1_df.iloc[i]['대분류']] = digit_1_df.iloc[i]['대분류명']

digit_2_dict = {}
for i in tqdm(range(len(digit_2_df))):
    digit_2_dict[int(digit_2_df.iloc[i]['중분류'])] = digit_2_df.iloc[i]['중분류명']

digit_3_dict = {}
for i in tqdm(range(len(digit_3_df))):
    digit_3_dict[int(digit_3_df.iloc[i]['소분류'])] = digit_3_df.iloc[i]['소분류명']

100%|██████████| 232/232 [00:00<00:00, 5155.22it/s]


In [114]:
mecab = Mecab()
tokenizer = Tokenizer()
stopwords = pd.read_csv("./data/stopwords/stopwords_concat.csv", encoding='CP949') 
stopwords = list(stopwords)
max_len = 10

In [146]:
def sentiment_predict(new_sentence):
  new_sentence = re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','', new_sentence)
  print(new_sentence)
  new_sentence = mecab.morphs(new_sentence) # 토큰화
  print(new_sentence)
  new_sentence = [word for word in new_sentence if not word in stopwords] # 불용어 제거
  print(new_sentence)
  encoded = tokenizer.texts_to_sequences([new_sentence]) # 정수 인코딩
  print(encoded)
  pad_new = pad_sequences(encoded, maxlen = max_len) # 패딩
  print(pad_new)
  label = loaded_model.predict(pad_new) # 예측
  print(label)
  print(digit_1_dict[le.inverse_transform([label.argmax()])[0]])

In [ ]:
구두가게에서 일반인을 대상으로 구두소매

In [138]:
digit_1_dict

{'A': '농업, 임업 및 어업(01~03)',
 'B': '광업(05~08)',
 'C': '제조업(10~34)',
 'D': '전기, 가스, 증기 및 공기 조절 공급업(35)',
 'E': '수도, 하수 및 폐기물 처리, 원료 재생업(36~39)',
 'F': '건설업(41~42)',
 'G': '도매 및 소매업(45~47)',
 'H': '운수 및 창고업(49~52)',
 'I': '숙박 및 음식점업(55~56)',
 'J': '정보통신업(58~63)',
 'K': '금융 및 보험업(64~66)',
 'L': '부동산업(68)',
 'M': '전문, 과학 및 기술 서비스업(70~73)',
 'N': '사업시설 관리, 사업 지원 및 임대 서비스업(74~76)',
 'O': '공공 행정, 국방 및 사회보장 행정(84)',
 'P': '교육 서비스업(85)',
 'Q': '보건업 및 사회복지 서비스업(86~87)',
 'R': '예술, 스포츠 및 여가관련 서비스업(90~91)',
 'S': '협회 및 단체, 수리 및 기타 개인 서비스업(94~96)',
 'T': '가구 내 고용활동 및 달리 분류되지 않은 자가 소비 생산활동(97~98)',
 'U': '국제 및 외국기관(99)'}

In [137]:
le.classes_

array(['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M',
       'N', 'O', 'P', 'Q', 'R', 'S'], dtype=object)

In [148]:
sentiment_predict('쌀을 가지고|가공하여|떡제조')

쌀을 가지고가공하여떡제조
['쌀', '을', '가지', '고', '가공', '하', '여', '떡', '제조']
['쌀', '을', '가지', '고', '가공', '하', '여', '떡', '제조']
[[9, 2, 10, 11, 12, 13, 14, 15, 16]]
[[ 0  9  2 10 11 12 13 14 15 16]]
[[1.08232035e-03 1.57818128e-03 2.46537682e-02 2.30564061e-03
  4.23182640e-03 1.50569575e-02 7.90182233e-01 1.19435318e-01
  1.20275719e-02 2.80325511e-03 4.15205344e-04 2.66796025e-03
  1.11436211e-02 4.01561148e-03 7.70625367e-04 8.72150646e-04
  1.85668922e-03 8.50530691e-04 4.05047927e-03]]


KeyError: 6

'숙박 및 음식점업(55~56)'

In [ ]:
le.inverse_transform([label.argmax()])[0]

In [119]:
label

array([[1.6976253e-04, 5.4305434e-05, 3.5700612e-04, 2.5740022e-05,
        3.6520050e-05, 8.6037413e-04, 7.2269826e-03, 3.5431760e-04,
        8.6225563e-01, 1.0106920e-03, 1.1226628e-03, 1.0101624e-03,
        1.8476601e-03, 2.4819549e-03, 1.7784047e-05, 2.4636678e-02,
        3.0528158e-03, 6.1916668e-02, 3.1562213e-02]], dtype=float32)

In [106]:
label

array([[0.12797637, 0.08861163, 0.08251524, 0.02988727, 0.06311161,
        0.02287801, 0.03186466, 0.02674469, 0.02942831, 0.02350585,
        0.05054095, 0.01838752, 0.01585915, 0.01778417, 0.04686501,
        0.12415843, 0.06648891, 0.08605903, 0.04733315]], dtype=float32)

In [ ]:
절에서 신도을 대상으로 불교단체운영

'I'

8

'농업, 임업 및 어업(01~03)'

'A'

array(['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M',
       'N', 'O', 'P', 'Q', 'R', 'S'], dtype=object)